# Improving Patch Classification
In this notebook, I'm going to explore how to improve the patch classifier as the current one is not really performing. In fact let's try out a bunch of the pretrained ones from torch vision. We have quite large datasets so it makes sense to try and train the whole network, not just the end layers.

In [1]:
# standard project preamble
from pathgen.utils.seeds import set_seed
from pathgen.utils.paths import project_root

experiment_name = "all"
experiment_root = project_root() / "experiments" / experiment_name

global_seed = 123
set_seed(global_seed)

In [2]:
# standard data science imports
import numpy as np

# standard pytorch imports
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

# pytorch data loading imports
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# torchvision
import torchvision
import torchvision.transforms as transforms

In [3]:
# define our hyper parameters
batch_size = 128
num_epochs = 1
learning_rate = 0.001

In [4]:
# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# set up the data loaders for training
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set = ImageFolder(experiment_root / 'train_patches', transform=transform)  # 70,000 samples (35,000 per class) - non-fake
valid_set = ImageFolder(experiment_root / 'valid_patches', transform=transform)  # 30,000 samples (15,000 per class) - non-fake

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, worker_init_fn=np.random.seed(global_seed), num_workers=32)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, worker_init_fn=np.random.seed(global_seed), num_workers=32)

In [6]:
# set up the network
from torchsummary import summary

# load the model
model = torchvision.models.vgg16(pretrained=True)
model.classifier[6] = nn.Sequential(nn.Linear(in_features=4096, out_features=1, bias=True))

# get a summary of the model based on our input shape
model.to(device)
img, _ = train_set[0]
img.to(device)
summary(model, img.shape, batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 256, 256]           1,792
              ReLU-2        [128, 64, 256, 256]               0
            Conv2d-3        [128, 64, 256, 256]          36,928
              ReLU-4        [128, 64, 256, 256]               0
         MaxPool2d-5        [128, 64, 128, 128]               0
            Conv2d-6       [128, 128, 128, 128]          73,856
              ReLU-7       [128, 128, 128, 128]               0
            Conv2d-8       [128, 128, 128, 128]         147,584
              ReLU-9       [128, 128, 128, 128]               0
        MaxPool2d-10         [128, 128, 64, 64]               0
           Conv2d-11         [128, 256, 64, 64]         295,168
             ReLU-12         [128, 256, 64, 64]               0
           Conv2d-13         [128, 256, 64, 64]         590,080
             ReLU-14         [128, 256,

## Saving and loading models
Let's add some functions for loading an saving the state of the model and training using a checkpoint.

In [7]:
def save_checkpoint(epoch, model, optimizer, loss, path="my_checkpoint.ckpt"):
    print("saving checkpoint")
    state = { 'epoch': epoch, 
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict() }
    torch.save(state, path)

def load_checkpoint(model, optimizer, path="my_checkpoint.ckpt"):
    print("loading checkpoint")
    state = torch.load(path)
    epoch = state["epoch"]
    model.load_state_dict(state["state_dict"])
    optimizer.load_state_dict(state["optimizer"])
    return epoch, model, optimizer, loss

## Training
Now that we have the data loader created and the model defined, let's define our loss and optimiser and run the training loop.

In [8]:
from statistics import mean

class Logger:
    def __init__(self):
        self.losses_per_epoch = []
        self.losses_current_epoch = []
        self.accuracy_per_epoch = []
        self.accuracy_current_epoch = []
    
    def loss(self, loss):
        self.losses_current_epoch.append(loss.item())
        
    def accuracy(self, acc):
        self.accuracy_current_epoch.append(acc.item())
    
    def next_epoch(self, epoch, should_print=False):
        mean_loss = mean(self.losses_current_epoch)
        self.losses_per_epoch.append(mean_loss)
        self.losses_current_epoch = []
        
        mean_accuracy = mean(self.accuracy_current_epoch)
        self.accuracy_per_epoch.append(mean_accuracy)
        self.accuracy_current_epoch = []
        
        if should_print:
            print(f"epoch {epoch} complete. loss: {mean_loss}, accuracy: {mean_accuracy}")

In [9]:
def accuracy(scores, targets):
    _, predictions = scores.max(1)
    num_correct = (predictions == targets).sum()
    num_samples = predictions.size(0)
    return num_correct / num_samples

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

In [10]:
%%timeit

log = Logger()

criterion = nn.BCEWithLogitsLoss()  # combines a Sigmoid layer and the BCELoss in one single class
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    print(f"starting epoch {epoch}")
    for batch_idx, (data, targets) in enumerate(train_loader):
        print(f'{batch_idx}', end=' ')
        
        # put X and y for the batch on the GPU is possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward pass
        scores = model(data)
        loss = criterion(scores, targets.unsqueeze(1).float())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()
        
        # log the metrics
        acc = binary_acc(scores, targets.unsqueeze(1))
        log.accuracy(acc)
        log.loss(loss)
    
    save_checkpoint(epoch, model, optimizer, experiment_root / f"vgg16_checkpoint_{epoch}.ckpt") 
    log.next_epoch(epoch, should_print=True)

starting epoch 0
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 2

Now let's visualise the training process and see how things went.

In [11]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(log.losses_per_epoch, label="training loss")
# plt.plot(valid_log.epoch, valid_log.val_loss, label="validation loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.title("Patch Classifier Real Data Loss")
plt.legend()

NameError: name 'log' is not defined

<Figure size 432x288 with 0 Axes>

Let's test the model on the test set to see how it performs.

In [ ]:
def accuracy(logits, y):
    # pred = torch.log_softmax(logits, dim=1)
    # correct = pred.argmax(dim=1).eq(y).sum().item()
    correct = logits.argmax(dim=1).eq(y).sum().item()
    total = len(y)
    accu = correct / total
    return accu

# x = self.softmax(self.fc3(x))